**IA and locomotion: human gait analysis**

# Introduction

**Context**

The study of human gait is a central problem in medical research with far-reaching consequences in the public health domain.
This complex mechanism can be altered by a wide range of pathologies (such as Parkinson’s disease, arthritis, stroke,...), often resulting in a significant loss of autonomy and an increased risk of fall.
Understanding the influence of such medical disorders on a subject's gait would greatly facilitate early detection and prevention of those possibly harmful situations.
To address these issues, clinical and bio-mechanical researchers have worked to objectively quantify gait characteristics.


Among the gait features that have proved their relevance in a medical context, several are linked to the notion of step (step duration, variation in step length, etc.), which can be seen as the core atom of the locomotion process.
Many algorithms have therefore been developed to automatically (or semi-automatically) detect gait events (such as heel-strikes, heel-off, etc.) from accelerometer/gyrometer signals.

Most of the time, the algorithms used for step detection are dedicated to a specific population (healthy subjects, elderly subjects, Parkinson patients, etc.) and only a few publications deal with heterogeneous populations composed of several types of subjects.
Another limit to existing algorithms is that they often focus on locomotion in established regime (once the subject has initiated its gait) and do not deal with steps during U-turn, gait initiation or gait termination.
Yet, initiation and termination steps are particularly sensitive to pathological states.
For example, the first step of Parkinsonian patients has been described as slower and smaller that the first step of age-matched subjects.
U-turn steps are also interesting since 45% of daily living walking is made up of turning steps, and when compared to straight-line walking, turning has been emphasized as a high-risk fall situation.
This argues for reliable algorithms that could detect initiation, termination and turning steps in both healthy and pathological subjects.


**Step detection**

The objective is to recognize the **start and end times of footsteps** contained in accelerometer and gyrometer signals recorded with Inertial Measurement Units (IMUs).

## Setup

**Import**

In [ ]:
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from loadmydata.load_human_locomotion import (get_code_list,
                                              load_human_locomotion_dataset)
from metric import fscore
from scipy.linalg import circulant
from scipy.signal import argrelmax
from scipy.stats import pearsonr
from sklearn.decomposition import SparseCoder
from sklearn.utils import Bunch

**Utility functions**

In [ ]:
def get_avg_min_max(a_list) -> str:
    """[a_1, a_2,...] -> 'avg (min: minimum, max: maximum)'"""
    return f"{np.mean(a_list):.1f} (min: {np.min(a_list):.1f}, max: {np.max(a_list):.1f})"

In [ ]:
def pad_with_zeros(signal: np.ndarray, desired_length: int) -> np.ndarray:
    """Add zeros at the start and end of a signal until it reached the desired lenght.

    The same number of zeros is added on each side, except when desired_length-signal.shape[0] is odd,
    in which case, there is one more zero at the beginning.
    """
    if signal.ndim == 1:
        (n_samples,) = signal.shape
        n_dims = 1
    else:
        n_samples, n_dims = signal.shape

    assert desired_length >= n_samples

    length_diff = desired_length - n_samples
    pad_width_at_the_start = pad_width_at_the_end = length_diff // 2
    pad_width_at_the_start += (
        length_diff - pad_width_at_the_end - pad_width_at_the_start
    )

    return np.pad(
        signal.reshape(n_samples, n_dims).astype(float),
        pad_width=((pad_width_at_the_start, pad_width_at_the_end), (0, 0)),
        mode="constant",
        constant_values=(0,),
    )

In [ ]:
def pad_at_the_end(signal: np.ndarray, desired_length: int) -> np.ndarray:
    """Add zeros at the end of a signal until it reached the desired length."""
    if signal.ndim == 1:
        (n_samples,) = signal.shape
        n_dims = 1
    else:
        n_samples, n_dims = signal.shape

    assert desired_length >= n_samples

    pad_width_at_the_end = desired_length - n_samples

    return np.pad(
        signal.reshape(n_samples, n_dims).astype(float),
        pad_width=((0, pad_width_at_the_end), (0, 0)),
        mode="constant",
        constant_values=(0,),
    )

In [ ]:
def fig_ax(figsize=(15, 3)):
    return plt.subplots(figsize=figsize)

In [ ]:
def get_sparse_codes(
    signal: np.ndarray, dictionary: np.ndarray, penalty: float
):
    coder = SparseCoder(
        dictionary=dictionary,
        transform_algorithm="lasso_lars",
        transform_alpha=penalty,
        positive_code=True,
    )
    return coder.transform(signal.reshape(1, -1))

In [ ]:
def get_dictionary_from_single_atom(atom, n_samples):
    atom_width = atom.shape[0]
    dictionary = circulant(pad_at_the_end(atom, n_samples).flatten())[
        :, : n_samples - atom_width + 1
    ].T
    return dictionary

In [ ]:
def plot_CDL(signal, codes, atoms, figsize=(15, 10)):
    """Plot the learned dictionary `D` and the associated sparse codes `Z`.

    `signal` is an univariate signal of shape (n_samples,) or (n_samples, 1).
    """
    (n_atoms, atom_length) = atoms.shape
    plt.figure(figsize=figsize)
    plt.subplot(n_atoms + 1, 3, (2, 3))
    plt.plot(signal)
    for i in range(n_atoms):
        plt.subplot(n_atoms + 1, 3, 3 * i + 4)
        plt.plot(atoms[i])
        plt.subplot(n_atoms + 1, 3, (3 * i + 5, 3 * i + 6))
        plt.plot(codes[i])
        plt.ylim((np.min(codes), np.max(codes)))

In [ ]:
def resample_to_size(signal, desired_size=100):
    return np.interp(
        np.arange(desired_size),
        np.linspace(0, desired_size, signal.shape[0]),
        signal,
    )

In [ ]:
def get_locogram(sensor_data, left_or_right="left"):
    if left_or_right == "left":
        steps = sensor_data.left_steps
        acc_columns = ["LAX", "LAY", "LAZ"]
    elif left_or_right == "right":
        steps = sensor_data.right_steps
        acc_columns = ["RAX", "RAY", "RAZ"]

    n_steps = steps.shape[0]
    locogram = np.zeros((n_steps, n_steps))

    acc_norm = np.linalg.norm(
        sensor_data.signal[acc_columns].to_numpy(), axis=1
    )

    for step_ind_1 in range(n_steps):
        start, end = steps[step_ind_1]
        step_1 = resample_to_size(acc_norm[start:end])
        for step_ind_2 in range(step_ind_1 + 1, n_steps):
            start, end = steps[step_ind_2]
            step_2 = resample_to_size(acc_norm[start:end])
            locogram[step_ind_1, step_ind_2] = pearsonr(step_1, step_2)[0]

    locogram += locogram.T
    np.fill_diagonal(a=locogram, val=1.0)

    return locogram

## Data loading

In [ ]:
# This wil download the data on the first run
_ = load_human_locomotion_dataset("1-1")

# Data description

## Data collection and clinical protocol

#### Participants

The data was collected between April 2014 and October 2015 by monitoring healthy (control) subjects and patients from several medical departments (see [publication](#Publication) for more information).
Participants are divided into three groups depending on their impairment:
- **Healthy** subjects had no known medical impairment.
- The **orthopedic group** is composed of 2 cohorts of distinct pathologies: lower limb osteoarthrosis and cruciate ligament injury.
- The **neurological group** is composed of 4 cohorts: hemispheric stroke, Parkinson's disease, toxic peripheral neuropathy and radiation induced leukoencephalopathy.

Note that certain participants were recorded on multiple occasions, therefore several trials may correspond to the same person.
In the training set and in the testing set, the proportion of trials coming from the "healthy", "orthopedic" and "neurological" groups is roughly the same, 24%, 24% and 52% respectively.

#### Protocol and equipment

All subjects underwent the same protocol described below. First, a IMU (Inertial Measurement Unit) that recorded accelerations and angular velocities was attached to each foot.
All signals have been acquired at 100 Hz with two brands of IMUs: XSens&trade; and Technoconcept&reg;.
One brand of IMU was attached to the dorsal face of each foot.
(Both feet wore the same brand.)
After sensor fixation, participants were asked to perform the following sequence of activities:
- stand for 6 s,
- walk 10 m at preferred walking speed on a level surface to a previously shown turn point,
- turn around (without previous specification of a turning side),
- walk back to the starting point,
- stand for 2 s.

Subjects walked at their comfortable speed with their shoes and without walking aid.
This protocol is schematically illustrated in the following figure.


<div style="text-align: center">
<img src="https://raw.githubusercontent.com/ramp-kits/human_locomotion/master/images/protocol-schema.png" width="500px">
</div>


Each IMU records its acceleration and angular velocity in the $(X, Y, Z, V)$ set of axes defined in the following figure.
The $V$ axis is aligned with gravity, while the $X$, $Y$ and $Z$ axes are attached to the sensor.
<div style="text-align: center">
<img src="https://raw.githubusercontent.com/ramp-kits/human_locomotion/master/images/sensor-photo.png" width="500px">
</div>

<div style="text-align: center">
<img src="https://raw.githubusercontent.com/ramp-kits/human_locomotion/master/images/sensor-position.png" width="500px">
</div>

## Step detection in a clinical context

The following schema describes how step detection methods are integrated in a clinical context.
<br/><br/>
<div style="text-align: center">
<img src="https://raw.githubusercontent.com/ramp-kits/human_locomotion/master/images/step-detection-schema.png" width="500px">
</div>

(1) During a trial, sensors send their own acceleration and angular velocity to the physician's computer.

(2) A software on the physician's computer synchronizes the data sent from both sensors and produces two multivariate signals (of same shape), each corresponding to a foot.


A step detection procedure is applied on each signal to produce two lists of footsteps (one per foot/sensor).
The numbers of left footsteps and right footsteps are not necessarily the same.
Indeed, subjects often have a preferred foot to initiate and terminate a walk or a u-turn, resulting in one or more footsteps from this preferred foot.
The starts and ends of footsteps are then used to create meaningful features to characterize the subject's gait.

## Data exploration

During a trial, a subject executes the protocol described above.
This produces two multivariates signals (one for each foot/sensor) and for each signal, a number of footsteps have be annotated.
In addition, information (metadata) about the trial and participant are provided.
All three elements (signal, step annotation and metadata) are detailled in this section.

In [ ]:
# chosing a trial
code_list = get_code_list()
code = code_list[90]
# loading a trial
sensor_data = load_human_locomotion_dataset(code)
# print data set description
print(sensor_data.description)

In [ ]:
# Uncomment any of the following line the show the attributes of `data`.

print(sensor_data.signal)  # pandas array
# print(sensor_data.left_steps)  # numpy array (n_left_steps, 2)
# print(sensor_data.right_steps)  # numpy array (n_right_steps, 2)
print(sensor_data.metadata)  # dictionary

### Signal

Each IMU that the participants wore provided $\mathbb{R}^{8}$-valued signals, sampled at 100 Hz.
In this setting, each dimension is defined by the foot (`L` for left, `R` for right), the signal type (`A` for acceleration, `R` for angular velocity) and the axis (`X`, `Y`, `Z` or `V`).
For instance, `RRX` denotes the angular velocity around the `X`-axis of the right foot.
Accelerations are given in $m/s^2$ and angular velocities, in $deg/s$.
The signal is available in the `.signal` attribute as a `Pandas` dataframe.

Note that this multivariate signal originates from a two sensors (one on each foot).

In [ ]:
# The signal is available in the `signal` attribute.

fig, (ax_0, ax_1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 3))

# Here we show the left foot (`L`)
sensor_data.signal[["LAX", "LAY", "LAZ", "LAV"]].plot(
    ax=ax_0
)  # select the accelerations
sensor_data.signal[["LRX", "LRY", "LRZ", "LRV"]].plot(
    ax=ax_1
)  # select the angular velocities

sensor_data.signal.head()

The "flat part" at the beginning of each dimension is the result of the participants standing still for a few
seconds before walking (see [Protocol](#Protocol-and-equipment)).
The same behaviour can be seen at the end of each dimension (often but not always), though for a quite smaller duration.

###  Metadata
A number of metadata (either numerical or categorical) are provided for each sensor recording, detailing the participant being monitored and the sensor position:

- `trial_code`: unique identifier for the trial;
- `age` (in years);
- `gender`: male ("M") or female ("F");
- `height` (in meters);
- `weight` (in kilograms);
- `bmi` (in kg/m2): body mass index;
- `laterality`: subject's "footedness" or "foot to kick a ball" ("Left", "Right" or "Ambidextrous").
- `sensor`: brand of the IMU used for the recording (“XSens” or “TCon”);
- `pathology_group`: this variable takes value in {“Healthy”, “Orthopedic”, “Neurological”};
- `is_control`: whether the subject is a control subject ("Yes" or "No");
- `foot`: foot on which the sensor was attached ("Left" or "Right").

These are accessible using the notation `sensor_data.metadata`.

In [ ]:
sensor_data.metadata

In [ ]:
trial_metadata = Bunch(**sensor_data.metadata)

msg = f"""Metadata:
age\t\t{trial_metadata.Age},
gender\t\t{trial_metadata.Gender},
height\t\t{trial_metadata.Height},
weight\t\t{trial_metadata.Height},
bmi\t\t{trial_metadata.BMI},
laterality\t{trial_metadata.Laterality},
sensor\t\t{trial_metadata.Sensor},
pathology_group\t{trial_metadata.PathologyGroup},
trial code\t{trial_metadata.Code}"""
print(msg)

### Step annotation (the "label" to predict)
Footsteps were manually annotated by specialists using a software that displayed the signals from the relevant sensor (left or right foot) and allowed the specialist to indicate the starts and ends of each step.

A footstep is defined as the period during which the foot is moving.
Footsteps are separated by periods when the foot is still and flat on the floor.
Therefore, in our setting, a footstep starts with a heel-off and ends with the following toe-strike of the same foot.


Footsteps (the "label" to predict from the signal) are contained in a list whose elements are list of two integers, the start and end indexes. For instance:

In [ ]:
# left foot
print(sensor_data.left_steps)

Visualization of footsteps and signals:

In [ ]:
msg = f"For the trial '{trial_metadata.Code}', {sensor_data.left_steps.shape[0]} footsteps were annotated on the left foot, and {sensor_data.right_steps.shape[0]} on the right."
print(msg)


# Color the footsteps
fig, (ax_0, ax_1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 3))

ax = ax_0
sensor_data.signal[["LAX", "LAY", "LAZ", "LAV"]].plot(ax=ax)
line_args = {"linestyle": "--", "color": "k"}
for (start, end) in sensor_data.left_steps:
    ax.axvline(start, **line_args)
    ax.axvline(end, **line_args)
    ax.axvspan(start, end, facecolor="g", alpha=0.3)

ax = ax_1
sensor_data.signal[["LRX", "LRY", "LRZ", "LRV"]].plot(ax=ax)
for (start, end) in sensor_data.left_steps:
    ax.axvline(start, **line_args)
    ax.axvline(end, **line_args)
    ax.axvspan(start, end, facecolor="g", alpha=0.3)


# Close-up on a footstep
fig, (ax_0, ax_1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 3))

start, end = sensor_data.left_steps[4]

ax = ax_0
sensor_data.signal[["LAX", "LAY", "LAZ", "LAV"]][start - 30 : end + 30].plot(
    ax=ax
)
ax.axvline(start, **line_args)
ax.axvline(end, **line_args)
ax.axvspan(start, end, facecolor="g", alpha=0.3)

ax = ax_1
sensor_data.signal[["LRX", "LRY", "LRZ", "LRV"]][start - 30 : end + 30].plot(
    ax=ax
)
ax.axvline(start, **line_args)
ax.axvline(end, **line_args)
_ = ax.axvspan(start, end, facecolor="g", alpha=0.3)

**On the first two plots.**
The repeated patterns (colored in light green) correspond to periods when the foot is moving.
During the non-annotated periods, the foot is flat and not moving and the signals are constant.
Generally, steps at the beginning and end of the recording, as well as during the u-turn (in the middle of the signal approximatively, see [Protocol](#Protocol-and-equipment)) are a bit different from the other ones.

**On the last two plots.** A close-up on a single footstep.

### General comments

- Some metadata (namely `Age`, `Height`, `Weight`, `BMI` and `Laterality`) can take the value "NC" which stands for "Not Communicated". This label replaces missing data and depending on the variable may affect up to 2% of the database.

- There are uncertainties in the definition of the starts and ends of the steps. Indeed, we can see on previous figures that the start and end could be slightly moved. However, our choice of metric is relatively immune to small variations in the start and end of footsteps.

- There is a lot of variability in the step patterns depending on the pathology, the age, the weight, the sensor brand, etc. We invite the participants to skim through the different trials to see how footsteps vary. Generally, long signals (over 40 seconds) display pathological behaviours.

- For a given trial, the two associated signals (left foot sensor and right foot sensor) have the same duration (and therefore the same shape). However they might not have the same number of annotated footsteps. Indeed, it often happens that one foot makes one step more compared to the other. Also, between trials, the number of signal samples greatly varies.

### Simple data exploration

In [ ]:
# signal duration
sampling_freq = 100  # Hz

duration_list = list()
for code in code_list:
    sensor_data = load_human_locomotion_dataset(code)
    duration_list += [sensor_data.signal.shape[0] / sampling_freq]

print(
    f"On average, a recording lasts {get_avg_min_max(duration_list)} seconds;"
)

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>Compute the following:</p>
    <ul>
        <li>the average number of steps per trial,</li>
        <li>the average foostep duration,</li>
        <li>the number of trials for each pathology.</li>
    </ul>
</div>

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>Which is the pathology group of the patient with the longuest trial?</p>
    <p>Plot its signal.</p>
    
</div>

We can show the locogram for the trial at hand.

In [ ]:
locogram = get_locogram(sensor_data=sensor_data, left_or_right="left")

In [ ]:
_ = sns.heatmap(1 - locogram)

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>Compare the left and right locograms.</p>
</div>

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>Compute the locograms for a trial with a few steps (between 10 and 15) and for a trial with many steps (more than 30).</p>
</div>

# Step detection

## Performance metric

Step detection methods will be evaluated with the **F-score**, based on the following precision/recall definitions.
The F-score is first computed per signal then averaged over all instances.

Precision and recall rely on the "intersection over union" metric ($\text{IoU}$) that measures the overlap of two intervals $[s_1,e_1]$ and $[s_2, e_2]$:

$$
\text{IoU}=\frac{\big|[s_1,e_1]\cap [s_2, e_2]\big|}{\big|[s_1,e_1]\cup [s_2, e_2]\big|}
$$

- Precision (or positive predictive value). A detected (or predicted) step is counted as correct if it overlaps (measured by $\text{IoU}$) an annotated step by more than 75%. The precision is the number of correctly predicted steps divided by the total number of predicted steps.

- Recall (or sensitivity). An annotated step is counted as detected if it overlaps (measured by $\text{IoU}$) a predicted step by more than 75%. The recall is the number of detected annotated steps divided by the total number of annotated steps.


The F-score is the geometric mean of the precision and recall: $$2\times\frac{\text{precision}\times\text{recall}}{\text{precision}+\text{recall}}.$$

Note that an annotated step can only be detected once, and a predicted step can only be used to detect one annotated step.
If several predicted steps correspond to the same annotated step, all but one are considered as false.
Conversely, if several annotated steps are detected with the same predicted step, all but one are considered undetected.

**Example 1.**

- Annotation ("ground truth label"): $\big[[80, 100], [150, 250], [260, 290]\big]$ (three steps)
- Prediction: $\big[[80, 98], [105, 120], [256, 295], [298, 310]\big]$ (four steps)

Here, precision is $0.5=(1+0+1+0)/4$, recall is $0.67=(1+0+1)/3$ and the F-score is $0.57$.

**Example 2.**

- Annotation ("ground truth label"): $\big[[80, 120]\big]$ (one step)
- Prediction: $\big[[80, 95]\big]$ (one step)

Here, precision is $0=0/1$, recall is $0=0/1$ and the F-score is $0$.

## Single atom

To illustrate, a simple step detection method is described now.
- A random step is chosen (the template).
- For the same signal, we perform a convolution sparse coding step.

In [ ]:
# choose a single dimension
signal = sensor_data.signal.LRY.to_numpy()
signal -= signal.mean()
signal /= signal.std()

n_samples = signal.shape[0]

# take an arbitrary footstep
start, end = sensor_data.left_steps[5]
template = signal[start:end]
template -= template.mean()
template /= template.std()

Transform the convolutional sparse coding into a regular sparse coding.

In [ ]:
dictionary = get_dictionary_from_single_atom(
    atom=template, n_samples=n_samples
)
sparse_codes = get_sparse_codes(
    signal=signal, dictionary=dictionary, penalty=30
)

In [ ]:
plot_CDL(
    signal=signal,
    codes=sparse_codes,
    atoms=template.reshape(1, -1),
    figsize=(15, 5),
)

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>What is the influence of the penalty on the number of non-zero activations?</p>
</div>

In [ ]:
fig, ax = fig_ax()
reconstruction = sparse_codes.dot(dictionary).flatten()
ax.plot(signal)
ax.plot(reconstruction)
ax.set_xlim(0, n_samples)
_ = ax.set_title(f"MSE = {((signal-reconstruction)**2).mean():.2f}")

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>What is the influence of the penalty on the reconstruction error?</p>
    <p>Is MSE a good measure to choose the penalty?</p>
</div>

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>How many steps were detected?</p>
</div>

We can look at the detected steps.

In [ ]:
start_arr = argrelmax(sparse_codes.flatten(), order=10)[0]
end_arr = start_arr + template.shape[0]
detected_steps = np.c_[start_arr, end_arr]

In [ ]:
fig, (ax_0, ax_1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 3))

# Color the footsteps
ax = ax_0
sensor_data.signal[["LAX", "LAY", "LAZ", "LAV"]].plot(ax=ax)
line_args = {"linestyle": "--", "color": "k"}
for (start, end) in detected_steps:
    ax.axvline(start, **line_args)
    ax.axvline(end, **line_args)
    ax.axvspan(start, end, facecolor="g", alpha=0.3)

ax = ax_1
sensor_data.signal[["LRX", "LRY", "LRZ", "LRV"]].plot(ax=ax)
for (start, end) in detected_steps:
    ax.axvline(start, **line_args)
    ax.axvline(end, **line_args)
    ax.axvspan(start, end, facecolor="g", alpha=0.3)

print(f"F-score: {fscore(sensor_data.left_steps, detected_steps):.2f}")

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>Using the `fscore`, choose an optimal value for the regularization parameter.</p>
</div>

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>Using the same template, detect steps in a whole new signal (try with the neurological group).</p>
    <p>What do you observe? What is your recommendation?</p>
</div>

In [ ]:
code = code_list[1000]
# loading a trial
sensor_data = load_human_locomotion_dataset(code)

In [ ]:
sensor_data.metadata